# 안산시 (1195)
- 작성자 : 박선빈
- 작성일자 : 2020년 11월 7일

In [1]:
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
from pathlib import Path
import numpy as np
import re
from tqdm import tqdm

### 1. Raw File 불러오기

In [2]:
dat ="2020-10-22" 
df = pd.read_csv(Path.cwd().parent / (dat.replace('-', '')[2:] + "_type1.csv"))
soup = BeautifulSoup(df[df.num == 1195].html.iloc[0], 'html.parser')

### 2. 간단한 전처리 하기

In [3]:
rows = soup.select('tr')
total = int(([rows.index(x) for x in rows if ">1</th>" in str(x)][0]+1)/2)

### 3. 저장할 DataFrame 미리 만들기

In [4]:
pat_col = ['num', 'dat', 'pth', 'syy', 'syd', 'cnd', 'rsd', 'atn', 'dch', 'adt'] 
rou_col = ['num', 'ord', 'dat', 'rgl', 'rgt', 'frm', 'exd', 'msk', 'ste', 'mob']

patient = pd.DataFrame(columns = pat_col)
route = pd.DataFrame(columns = rou_col)

### 4. 귀찮은 전처리해주기

In [5]:
for i in tqdm(range(total)):
    indiv1 = rows[2*i+1].find('th').text
    indiv = rows[2*i+1].select('td.blue')
    place = rows[2*i].text
    
    num = '1189' + '0' * (5- len(indiv1)) + indiv1
    pth = indiv[1].text; rsd = indiv[2].text; atn = indiv[3].text
    dch=0;    
    if '퇴원' in atn or '퇴소' in atn:
        dch=1; 
    syy=''; syd = ''; cnd =''; adt='';
    if len(place.split("\xa0")[-1].split('❍ ')) ==4:
        adt = place.split("\xa0")[-1].split('❍ ')[2].split(". ")[1]
        adt = [x for x in place.split("\xa0")[-1].split("❍ ") if "접촉자" in x][0].split(". ")[1]
    
    new_pat = pd.DataFrame([(num, dat, pth, syy, syd, cnd, rsd, atn, dch, adt)], columns = pat_col)
    patient = pd.concat([patient, new_pat])
    
    if '\xa0\xa0' in place:
        a=place.split("\xa0"); alist = list(filter(None, a)); alist = [v for v in alist if v != ' ']
        index = [index for index, x in enumerate(alist) if '소독' in x] 
        
        msk=''; mob=''; ste=1;
        for ind, j in enumerate(index[:-1]):
            ord = ind+1;
            rgl = alist[j].split('소독')[1][2:] + alist[j+1] + alist[j+4]; 
            rgt = alist[j+2]; frm =  alist[j+3]; exd = alist[j+5]
            new_route = pd.DataFrame(columns = rou_col, data = [[num, ord, dat, rgl, rgt, frm, exd, '', ste, '']])
            route = pd.concat([route, new_route])
        
    else:
        new_route = pd.DataFrame(columns = rou_col, data = [[num, 1, dat, '', '', '', '', '', '', '']])
        route = pd.concat([route, new_route])
    

100%|██████████| 33/33 [00:00<00:00, 90.23it/s]


### 5. 결과 잘 나왔나 확인하기

In [6]:
patient

,num,dat,pth,syy,syd,cnd,rsd,atn,dch,adt
0,11890000-,2020-10-22,해외입국자,,,,2020-10-18,이천생활치료센터,0,
0,118900032,2020-10-22,해외입국 외국인,,,,2020-10-17,안산생활치료센터,0,
0,118900031,2020-10-22,해외입국외국인,,,,2020-10-10,안산생활치료센터,0,"접촉자 4명(동반입국 3명-음성/자가격리, 1명-타시 이관/자가격리))"
0,118900030,2020-10-22,해외입국외국인,,,,2020-10-06,안산생활치료센터,0,접촉자 1명 자가격리
0,118900029,2020-10-22,해외입국외국인,,,,2020-09-30,퇴원,1,
0,118900028,2020-10-22,평택시 접촉자,,,,2020-09-26,퇴원,1,
0,118900027,2020-10-22,해외입국외국인,,,,2020-09-20,퇴소,1,
0,118900026,2020-10-22,미상,,,,2020-09-19,경기도의료원 안성병원,0,
0,118900025,2020-10-22,시흥시접촉자,,,,2020-09-17,퇴원,1,
0,118900024,2020-10-22,이천시 접촉자,,,,2020-09-15,퇴원,1,


In [7]:
route

,num,ord,dat,rgl,rgt,frm,exd,msk,ste,mob
0,11890000-,1,2020-10-22,,,,,,,
0,118900032,1,2020-10-22,경기도 안성시 대덕면,자택,-,10.16 20:30~10.17.15:00,,1,
0,118900032,2,2020-10-22,경기도 안성시 도기동,보건소,-,10.17 15:50,,1,
0,118900032,3,2020-10-22,경기도 안성시 금광면,단기격리시설,-,10.17 16:20~10.18,,1,
0,118900031,1,2020-10-22,경기도 안성시 대덕면,자택,-,10.16 18:00~19:45,,1,
0,118900031,2,2020-10-22,경기도 안성시 금광면,단기격리시설,-,10.16 20:15~10.18,,1,
0,118900030,1,2020-10-22,경기도 안성시 대덕면,자택,-,10.7~10.10,,1,
0,118900029,1,2020-10-22,,,,,,,
0,118900028,1,2020-10-22,,,,,,,
0,118900027,1,2020-10-22,,,,,,,
